In [28]:
import torch
import torch.nn.functional as F
from torch import nn

下面的CenteredLayer类要从其输入中减去均值。

In [29]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。

In [30]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到更复杂的模型中

In [31]:
net = nn.Sequential(nn.Linear(8, 128),
                    CenteredLayer())

In [32]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

带参数的层
下面我们继续定义具有参数的层，这些参数可以通过训练进行调整。
现在，我们实现自定义版的全连接层。

In [33]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化MyLinear类并访问其参数模型

In [34]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.0993, -0.3349,  1.1735],
        [ 0.3159,  0.0339, -0.2999],
        [-1.1208, -2.3258,  3.3634],
        [-0.3416,  1.8590, -0.4753],
        [ 1.3246,  0.8555,  0.2481]], requires_grad=True)

我们可以使用自定义层直接执行向前传播计算

In [36]:
linear(torch.rand(4, 5))

tensor([[0.0000, 0.3550, 1.2297],
        [0.0000, 0.0000, 2.4941],
        [0.0000, 0.2608, 1.5669],
        [0.1392, 2.1161, 0.0000]])

我们可以使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层

In [38]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(7, 64))

tensor([[ 3.6987],
        [ 3.4555],
        [ 2.3213],
        [ 0.4562],
        [ 0.0000],
        [ 0.9004],
        [10.7204]])

小结：
1 通过基本层类设计自定义层。这允许我们灵活的新层
2 在自定义完成后，我们可以在任意环境和网络架构中定义该层
3 层可以有局部参数，这些参数可以通过内置函数创建